In [5]:
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from matplotlib.pyplot import *
import os

plotly.tools.set_credentials_file(username='pusiol', api_key='m6iurl7f89')

In [6]:
filename = 'output_thermix_7.file'
base_dir = '/media/Gui2/thermix/trained_models/thermix_7_26-05-2016'

In [9]:
with open(os.path.join(base_dir,filename),'r') as f:
    # get all lines in file
    lines = f.readlines()

    # remove all those weird chars
    newlines = []
    for line in lines:
        newline = line.replace('\x08',' ')
        newlines.append(filter(None, newline.split('  ')))

    #now search for 75 / 75 (training) y 16 / 16 (validation)
    training_error_text = '42 / 42'
    validation_error_text = '8 / 8'

    training_error_lines = []
    validation_error_lines = []
    for a in newlines:
        for s in a:
            if training_error_text in s:
                training_error_lines.append(s)
            if validation_error_text in s:
                validation_error_lines.append(s)

    # get matrix for validation loss
    v_loss = []
    for v_data in validation_error_lines:
        index_rate = v_data.index('rate')
        v_loss.append(float(v_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))

    # get matrix for training loss
    t_loss = []
    for t_data in training_error_lines:
        index_rate = t_data.index('rate')
        t_loss.append(float(t_data[index_rate+4:index_rate+10].replace('%','').replace(' ','')))


In [10]:
#plot data
data = []
Ys = [('validation sparse softmax cross entropy loss', v_loss), ('training sparse softmax cross entropy loss', t_loss)]
for title, _Y in Ys:
    # t = 1 -> validation data
    _X = range(1,len(_Y)+1) # _X = epoch number
    mom = Scatter(x=_X,y=_Y, mode ='lines+markers', name =title ,line = dict(width = 4))
    data.append(mom)

layout = Layout(title='%s loss function'%filename.split(".")[0], xaxis = dict(title = 'Epoch'), yaxis = dict(title = 'miss rate'), )

# Plot and embed in ipython notebook!
fig = dict(data=data, layout=layout)

#upload to guido's account
py.iplot(fig, filename=filename)
